***GENERATED CODE FOR testtimeseries PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
import pyspark
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = transformationDF.drop('DateofService')
        transformationDF = transformationDF.drop('TransactionDate')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
from fbprophet import Prophet
import pandas as pd


def driverProphet(df):
    df.columns = ['ds', 'y']
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run testtimeseriesHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT \r\n\r\n\t\t\tsub2.ChargeDetailID, \r\n\t\t\tsub2.DateofService,sub2.TransactionDate,Sub1.ClaimSubmitDate,\r\n\t\t\tCASE WHEN Sub1.FirstReasonDate IS NOT NULL THEN DATEDIFF(day, Sub1.ClaimSubmitDate, Sub1.FirstReasonDate) ELSE 0 END as days_to_firstdenial\r\n\r\nFROM \r\n(\r\n\t\t\tSELECT\r\n\t\t\tDISTINCT\r\n\t\t\tcd.ChargeDetailID, \r\n\t\t\tcd.DateofService, TransactionDate,tr.ReasonDate\r\n\t\r\nfrom ChargeDetail cd\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID  \r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = t.PayerID \r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID and tr.PayerId = t.PayerID\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\ninner join Payer py WITH (NOLOCK) on py.PayerID = t.PayerID\r\nleft join Payer py1 WITH (NOLOCK) on py1.PayerID = cd.SecondaryPayerID\r\nleft join Payer py2 WITH (NOLOCK) on py2.PayerID = cd.TertiaryPayerID\r\ninner join ServiceItem si WITH (NOLOCK) on cd.ServiceItemID = si.ServiceItemID\r\nleft outer join ServiceLocation sl WITH (NOLOCK) on cd.ServiceLocationID = sl.ServiceLocationID\r\ninner join ContractLinksPA clpa WITH (NOLOCK) on cd.PayerID = clpa.payer_id and cd.ProviderID = clpa.provider_id\r\ninner join ContractsPA copa WITH (NOLOCK) on clpa.contract_id = copa.contract_id\r\n\r\nWHERE cd.Amount >.01 and    e.IsBillable = 'true'\r\nAND CloseDate is not null and transactiontypeid=4\r\nAND TransactionDate between '1/01/2020' and '9/30/2020'\r\n) as sub2\r\n\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, MIN(TransactionDate) as ClaimSubmitDate,\r\n\t\t\t MIN(tr1.Reasondate) as FirstReasonDate\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tinner join TransactionReason tr1 WITH (NOLOCK) on tr1.ChargeDetailID = t.ChargeDetailID\r\n\t\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr1.TransactionReasonCodeID\r\n\t\tWHERE t.Amount > 0.1 and trc.ReasonType!='Other'\r\n\t\tgroup by t.ChargeDetailID\r\n) as sub1\r\n\r\n on sub1.cid = sub2.ChargeDetailID \t\r\n Group by   \r\n \t\t\tsub2.ChargeDetailID,sub2.DateofService,sub2.TransactionDate,\r\n\t\t\tClaimSubmitDate,FirstReasonDate", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run testtimeseriesHooks.ipynb
try:
	#transformationPreExecutionHook()

	timeseriesfe = TransformationTimeSeriesForecastingMain.run(sqlserver, json.dumps( {"FE": {"functionList": [{"function": "correlation"}], "stage_attributes": {"source_attributes": {"host": "wsc-sqlserver1.database.windows.net", "port": "1433", "password": "G5gTW5rnKVeAZWXm", "database": "NextGen_BOND_CLINIC", "optionalDB": "NextGen_BOND_CLINIC", "user": "Numtra", "qtype": "query", "query": "SELECT DISTINCT \r\n\r\n\t\t\tsub2.ChargeDetailID, \r\n\t\t\tsub2.DateofService,sub2.TransactionDate,Sub1.ClaimSubmitDate,\r\n\t\t\tCASE WHEN Sub1.FirstReasonDate IS NOT NULL THEN DATEDIFF(day, Sub1.ClaimSubmitDate, Sub1.FirstReasonDate) ELSE 0 END as days_to_firstdenial\r\n\r\nFROM \r\n(\r\n\t\t\tSELECT\r\n\t\t\tDISTINCT\r\n\t\t\tcd.ChargeDetailID, \r\n\t\t\tcd.DateofService, TransactionDate,tr.ReasonDate\r\n\t\r\nfrom ChargeDetail cd\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID  \r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = t.PayerID \r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID and tr.PayerId = t.PayerID\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\ninner join Payer py WITH (NOLOCK) on py.PayerID = t.PayerID\r\nleft join Payer py1 WITH (NOLOCK) on py1.PayerID = cd.SecondaryPayerID\r\nleft join Payer py2 WITH (NOLOCK) on py2.PayerID = cd.TertiaryPayerID\r\ninner join ServiceItem si WITH (NOLOCK) on cd.ServiceItemID = si.ServiceItemID\r\nleft outer join ServiceLocation sl WITH (NOLOCK) on cd.ServiceLocationID = sl.ServiceLocationID\r\ninner join ContractLinksPA clpa WITH (NOLOCK) on cd.PayerID = clpa.payer_id and cd.ProviderID = clpa.provider_id\r\ninner join ContractsPA copa WITH (NOLOCK) on clpa.contract_id = copa.contract_id\r\n\r\nWHERE cd.Amount >.01 and    e.IsBillable = 'true'\r\nAND CloseDate is not null and transactiontypeid=4\r\nAND TransactionDate between '1/01/2020' and '9/30/2020'\r\n) as sub2\r\n\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, MIN(TransactionDate) as ClaimSubmitDate,\r\n\t\t\t MIN(tr1.Reasondate) as FirstReasonDate\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tinner join TransactionReason tr1 WITH (NOLOCK) on tr1.ChargeDetailID = t.ChargeDetailID\r\n\t\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr1.TransactionReasonCodeID\r\n\t\tWHERE t.Amount > 0.1 and trc.ReasonType!='Other'\r\n\t\tgroup by t.ChargeDetailID\r\n) as sub1\r\n\r\n on sub1.cid = sub2.ChargeDetailID \t\r\n Group by   \r\n \t\t\tsub2.ChargeDetailID,sub2.DateofService,sub2.TransactionDate,\r\n\t\t\tClaimSubmitDate,FirstReasonDate", "dbtype": "mssql", "is_header": "Use Header Line", "server_url": "/numtraPlatform/NumtraPlatformV2/uploads/platform/"}}, "featureList": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "12518", "mean": "", "stddev": "", "min": "00015C23-72C2-41C9-842C-1D3582B34B4F", "max": "FFF931FB-0800-4F12-8A8D-62D64EE4F03C", "missing": "0", "distinct": "12518"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{}], "feature": "DateofService", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DateofService"}, {"transformationsData": [{}], "feature": "TransactionDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TransactionDate"}, {"transformationsData": [{}], "feature": "ClaimSubmitDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ClaimSubmitDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_to_firstdenial", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12518", "mean": "15.19", "stddev": "15.92", "min": "3", "max": "368", "missing": "0"}, "updatedLabel": "days_to_firstdenial"}], "features": {"timecolumn": "ClaimSubmitDate", "tocompare": "days_to_firstdenial"}, "dataPercentage": "10", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(timeseriesfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run testtimeseriesHooks.ipynb
try:
	#mlPreExecutionHook()

	model = driverProphet(timeseriesfe)

	#mlPostExecutionHook(model)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:

try:
        future = model.make_future_dataframe(periods=30,freq='MS',include_history=False)
        predictedDataFrame = model.predict(future)
        display(model.plot(predictedDataFrame))
        display(model.plot_components(predictedDataFrame))
        display(predictedDataFrame)
except Exception as ex:
    logging.error(ex)

